In [35]:
import numpy as np
from numpy.random import normal
from numpy.random import uniform
from numpy.linalg import inv
from math import sqrt
import pandas as pd

In [28]:
N, p, sigma, low, high = 1000, 3, 1, -1, 1
b = np.array([[2, 5, -4, 0]]).T

In [29]:
sigma_squared_hat_biased = []
sigma_squared_hat_unbiased = []
for i in range(10000):
    X = uniform(low, high, size=(N, p+1))
    X[:,0] = 1
    y = X.dot(b) + normal(0, sigma, (N, 1))
    b_hat = inv(X.T.dot(X)).dot(X.T).dot(y)
    y_hat = X.dot(b_hat)
    sse = np.sum((y - y_hat)**2)
    sigma_squared_hat_biased.append(sse / N)
    sigma_squared_hat_unbiased.append(sse / (N-p-1))

In [30]:
np.mean(sigma_squared_hat_biased), np.mean(sigma_squared_hat_unbiased)

(0.9966489911548544, 1.0006515975450345)

In [31]:
sigma_hat = np.sqrt(sigma_squared_hat_unbiased[-1])
V = inv(X.T.dot(X))
v = []
z = []
for j in range(p+1):
    z.append(b_hat[j][0] / sigma_hat / sqrt(V[j,j]))
    v.append(V[j,j])
v = np.array(v)

In [32]:
z

[62.076783701539874, 89.88906350920139, -73.71963246356712, 1.6633302387987412]

In [33]:
b_hat = np.squeeze(b_hat)

In [34]:
(b_hat - 1.96*np.sqrt(v)*sigma_hat), (b_hat + 1.96*np.sqrt(v)*sigma_hat)

(array([ 1.92735484,  4.81013028, -4.09824548, -0.01636925]),
 array([ 2.05303074,  5.02457258, -3.88596747,  0.19992334]))

In [36]:
data = pd.read_csv("Prostate Cancer.txt") 

In [99]:
names = ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']
a = np.array(data[names].values)
mu = np.mean(a, axis=0)
sigma = np.zeros(shape=mu.shape)
for i in range(len(mu)):
    sigma[i] = np.sqrt(np.sum((a[:,i] - mu[i])**2) / len(mu))

In [95]:
corr = np.zeros(shape=(len(names), len(names)))

In [96]:
for i in range(1, len(names)):
    for j in range(i):
        corr[i, j] = np.mean((a[:,i] - mu[i])*(a[:,j] - mu[j])) / (sigma[i] * sigma[j])

In [97]:
corr

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.02024381,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.0162371 ,  0.02511117,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.00197369,  0.03191599,  0.02527115,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.03888572,  0.01121334,  0.00849079, -0.00619487,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.04873375,  0.01187381,  0.00921314, -0.00050511,  0.04857503,
         0.        ,  0.        ,  0.        ],
       [ 0.03120535,  0.00410489,  0.01940455,  0.00561591,  0.02312253,
         0.03715268,  0.        ,  0.        ],
       [ 0.03129449,  0.00774718,  0.01992564,  0.00566206,  0.03302612,
         0.0455742 ,  0.05426115,  0.        ]])

In [100]:
np.mean((a[:,0] - mu[0])*(a[:,1] - mu[1])) / (sigma[0] * sigma[1])

0.02313578391833674